In [1]:
import torch
import torchinfo
import clip  # Import CLIP library
from torch.utils.data import DataLoader

from level_1_dataloader import image_dataloader
from level_1_classifier_1 import level_1_classifier, level_1_model, level_1_output_layer
from level_2_classifier import level_2_classifier, level_2_output_layer, level_2_pre_model_concate, level_2_post_model
from Trainer_1 import Trainer_level_1, Trainer_level_2
from Tester_1 import Tester_level_1, Tester_level_2

import copy

In [2]:
train_set = image_dataloader(csv_file='train_set.xls', root_dir='coverpage/coverpage/')
val_set = image_dataloader(csv_file='val_set.xls', root_dir='coverpage/coverpage/')
test_set = image_dataloader(csv_file='test_set.xls', root_dir='coverpage/coverpage/')


In [3]:
print('no. of training sample', len(train_set))
print('no. of validation sample', len(val_set))
print('no. of testing sample', len(test_set))

no. of training sample 22340
no. of validation sample 2804
no. of testing sample 2809


In [4]:
train_dataloader = DataLoader(train_set, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_set, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_set, batch_size=32, shuffle=True)

In [5]:
pip install transformers pillow

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import torch
from transformers import LlavaForConditionalGeneration, AutoProcessor
from torchinfo import summary
from PIL import Image
import requests

class VisionOnlyModel(torch.nn.Module):
    def __init__(self, vision_model):
        super(VisionOnlyModel, self).__init__()
        self.vision_model = vision_model

    def forward(self, pixel_values):
        # Extract the class token (first token) from the last hidden state
        outputs = self.vision_model(pixel_values=pixel_values).last_hidden_state
        class_token_output = outputs[:, 0, :]  # Take the first token
        return class_token_output

def fine_tune_load_image_model():
    # Load the pre-trained LLAVA model
    llava_model = LlavaForConditionalGeneration.from_pretrained("llava-hf/llava-1.5-7b-hf")

    # Extract the vision model
    vision_model = llava_model.vision_model

    # Wrap it in a new model class
    vision_only_model = VisionOnlyModel(vision_model)

    # Enable gradients for the image model's parameters
    for p in vision_only_model.parameters():
        p.requires_grad = True

    return vision_only_model

In [ ]:
vision_model = fine_tune_load_image_model()

In [6]:
from transformers import LlavaForConditionalGeneration

# Load the pre-trained LLAVA model
llava_model = LlavaForConditionalGeneration.from_pretrained("llava-hf/llava-1.5-7b-hf")

# Print the model structure to find the vision model
print(llava_model)


/home/prolay/anaconda3/envs/torch_2.1.0/lib/python3.10/site-packages/transformers/models/llava/configuration_llava.py:100: FutureWarning: The `vocab_size` argument is deprecated and will be removed in v4.42, since it can be inferred from the `text_config`. Passing this argument has no effect
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

LlavaForConditionalGeneration(
  (vision_tower): CLIPVisionModel(
    (vision_model): CLIPVisionTransformer(
      (embeddings): CLIPVisionEmbeddings(
        (patch_embedding): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
        (position_embedding): Embedding(577, 1024)
      )
      (pre_layrnorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (encoder): CLIPEncoder(
        (layers): ModuleList(
          (0-23): 24 x CLIPEncoderLayer(
            (self_attn): CLIPAttention(
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (layer_norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (mlp): CLIPMLP(
              (activation_fn): Quick

In [7]:
import torch
from transformers import LlavaForConditionalGeneration, AutoProcessor
from torchinfo import summary
from PIL import Image
import requests

class VisionOnlyModel(torch.nn.Module):
    def __init__(self, vision_model):
        super(VisionOnlyModel, self).__init__()
        self.vision_model = vision_model

    def forward(self, pixel_values):
        # Extract the class token (first token) from the last hidden state
        outputs = self.vision_model(pixel_values=pixel_values).last_hidden_state
        class_token_output = outputs[:, 0, :]  # Take the first token
        return class_token_output

def fine_tune_load_image_model():
    # Load the pre-trained LLAVA model
    llava_model = LlavaForConditionalGeneration.from_pretrained("llava-hf/llava-1.5-7b-hf")

    # Extract the vision model
    vision_model = llava_model.vision_tower.vision_model

    # Wrap it in a new model class
    vision_only_model = VisionOnlyModel(vision_model)

    # Enable gradients for the image model's parameters
    for p in vision_only_model.parameters():
        p.requires_grad = True

    return vision_only_model


In [8]:
vision_model = fine_tune_load_image_model()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [9]:
FE_model = fine_tune_load_image_model()
print(FE_model)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

VisionOnlyModel(
  (vision_model): CLIPVisionTransformer(
    (embeddings): CLIPVisionEmbeddings(
      (patch_embedding): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
      (position_embedding): Embedding(577, 1024)
    )
    (pre_layrnorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-23): 24 x CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (layer_norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=1024, out_features=4096, bias=

In [10]:
summary(
    vision_model, 
    input_size=(1, 3, 336, 336), 
    col_names=('input_size', 'output_size', 'num_params', 'kernel_size', 'mult_adds'), 
    verbose=1
)

Layer (type:depth-idx)                                       Input Shape               Output Shape              Param #                   Kernel Shape              Mult-Adds
VisionOnlyModel                                              [1, 3, 336, 336]          [1, 1024]                 --                        --                        --
├─CLIPVisionTransformer: 1-1                                 --                        [1, 1024]                 --                        --                        --
│    └─CLIPVisionEmbeddings: 2-1                             [1, 3, 336, 336]          [1, 577, 1024]            1,024                     --                        --
│    │    └─Conv2d: 3-1                                      [1, 3, 336, 336]          [1, 1024, 24, 24]         602,112                   [14, 14]                  346,816,512
│    │    └─Embedding: 3-2                                   [1, 577]                  [1, 577, 1024]            590,848                   --   

Layer (type:depth-idx)                                       Input Shape               Output Shape              Param #                   Kernel Shape              Mult-Adds
VisionOnlyModel                                              [1, 3, 336, 336]          [1, 1024]                 --                        --                        --
├─CLIPVisionTransformer: 1-1                                 --                        [1, 1024]                 --                        --                        --
│    └─CLIPVisionEmbeddings: 2-1                             [1, 3, 336, 336]          [1, 577, 1024]            1,024                     --                        --
│    │    └─Conv2d: 3-1                                      [1, 3, 336, 336]          [1, 1024, 24, 24]         602,112                   [14, 14]                  346,816,512
│    │    └─Embedding: 3-2                                   [1, 577]                  [1, 577, 1024]            590,848                   --   

In [11]:
import os

used_model = 'Llava'
used_model_feature_size = 1024
level_1_checkpoint_dir = './checkpoints/' + used_model + '/level_1/'
level_1_model_file = 'model.pth'
level_1_csvlogger_file = 'log.csv'
level_1_weights_path = os.path.join(level_1_checkpoint_dir, level_1_model_file)
feature_size_extract_from_level_1 = 64
no_epoch = 10
lr = 0.00001
optimizer = 'AdamW'

# Create the directory if it doesn't exist
os.makedirs(level_1_checkpoint_dir, exist_ok=True)

# Print a message to confirm the directory creation (optional)
print(f"Directory {level_1_checkpoint_dir} is ready.")

Directory ./checkpoints/Llava/level_1/ is ready.


In [12]:
used_model_feature_size = 1024  # Example value, adjust based on your model
feature_size_extract_from_level_1 = 1024 

In [13]:
# Define and prepare model components
classifier_level_1 = level_1_classifier(feature_size=used_model_feature_size, feature_size_extract_from_level_1=feature_size_extract_from_level_1)
output_layer_level_1 = level_1_output_layer(feature_size_extract_from_level_1=feature_size_extract_from_level_1, no_class=2)
model_level_1 = level_1_model(FE_model, classifier_level_1, output_layer_level_1)

In [14]:
torchinfo.summary(model_level_1, (1, 3, 336, 336), col_names=('input_size', 'output_size', 'num_params', 'kernel_size', 'mult_adds'), verbose=1)


Layer (type:depth-idx)                                            Input Shape               Output Shape              Param #                   Kernel Shape              Mult-Adds
level_1_model                                                     [1, 3, 336, 336]          [1, 2]                    --                        --                        --
├─VisionOnlyModel: 1-1                                            [1, 3, 336, 336]          [1, 1024]                 --                        --                        --
│    └─CLIPVisionTransformer: 2-1                                 --                        [1, 1024]                 --                        --                        --
│    │    └─CLIPVisionEmbeddings: 3-1                             [1, 3, 336, 336]          [1, 577, 1024]            1,193,984                 --                        347,407,360
│    │    └─LayerNorm: 3-2                                        [1, 577, 1024]            [1, 577, 1024]            2

/home/prolay/anaconda3/envs/torch_2.1.0/lib/python3.10/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


Layer (type:depth-idx)                                            Input Shape               Output Shape              Param #                   Kernel Shape              Mult-Adds
level_1_model                                                     [1, 3, 336, 336]          [1, 2]                    --                        --                        --
├─VisionOnlyModel: 1-1                                            [1, 3, 336, 336]          [1, 1024]                 --                        --                        --
│    └─CLIPVisionTransformer: 2-1                                 --                        [1, 1024]                 --                        --                        --
│    │    └─CLIPVisionEmbeddings: 3-1                             [1, 3, 336, 336]          [1, 577, 1024]            1,193,984                 --                        347,407,360
│    │    └─LayerNorm: 3-2                                        [1, 577, 1024]            [1, 577, 1024]            2

In [15]:
trainer = Trainer_level_1(model = model_level_1, level=1, training_dataloader = train_dataloader, validation_dataloader = val_dataloader, epoch = no_epoch, learning_rate = lr, use_gpu = True, opt_method = optimizer, checkpoint_dir = level_1_checkpoint_dir, checkpoint_filename=level_1_model_file, csv_logger = level_1_csvlogger_file  )

In [ ]:
trainer.run()

Finish initializing...
2024-06-11 13:49:51.135425
EPOCH 1:


  1%|▏                        | 4/699 [06:08<18:02:29, 93.45s/batch, loss=0.693]